In [1]:
import pandas as pd

# Load the air quality merged data
aq_df = pd.read_csv("../data/processed/AQ_merged_data_export.csv")

# Display basic info
print("Shape:", aq_df.shape)
print("Columns:", aq_df.columns.tolist())
print("\nHead:")
print(aq_df.head())

# Summary statistics
print("\nSummary statistics:")
print(aq_df.describe(include='all'))

# Check for missing values
print("\nMissing values per column:")
print(aq_df.isnull().sum())

Shape: (14767993, 9)
Columns: ['Country', 'Samplingpoint', 'Pollutant', 'Start', 'End', 'Value', 'Unit', 'AggType', 'Notation']

Head:
  Country           Samplingpoint  Pollutant                Start  \
0      NO  NO/SPO_NO0073A_38_1785         38  2019-01-01 00:00:00   
1      NO  NO/SPO_NO0073A_38_1785         38  2019-01-01 01:00:00   
2      NO  NO/SPO_NO0073A_38_1785         38  2019-01-01 02:00:00   
3      NO  NO/SPO_NO0073A_38_1785         38  2019-01-01 03:00:00   
4      NO  NO/SPO_NO0073A_38_1785         38  2019-01-01 04:00:00   

                   End   Value    Unit AggType Notation  
0  2019-01-01 01:00:00 -9900.0  ug.m-3    hour     PM10  
1  2019-01-01 02:00:00 -9900.0  ug.m-3    hour     PM10  
2  2019-01-01 03:00:00 -9900.0  ug.m-3    hour     PM10  
3  2019-01-01 04:00:00 -9900.0  ug.m-3    hour     PM10  
4  2019-01-01 05:00:00 -9900.0  ug.m-3    hour     PM10  

Summary statistics:
         Country          Samplingpoint     Pollutant                Start  \
cou

In [5]:
import pandas as pd
import numpy as np

# Load the merged air quality data
aq_df = pd.read_csv("../data/processed/AQ_merged_data_export.csv")

# Drop unnecessary columns
aq_cleaned = aq_df.drop(columns=['Samplingpoint', 'Pollutant', 'AggType'], errors='ignore')

# Keep only the 'End' column for datetime
aq_cleaned = aq_cleaned.rename(columns={'End': 'Datetime'})
if 'Start' in aq_cleaned.columns:
    aq_cleaned = aq_cleaned.drop(columns=['Start'])

# Convert 'Datetime' to pandas datetime
aq_cleaned['Datetime'] = pd.to_datetime(aq_cleaned['Datetime'], errors='coerce')
aq_cleaned = aq_cleaned.dropna(subset=['Datetime'])

# Extract date and hour for grouping
aq_cleaned['Date'] = aq_cleaned['Datetime'].dt.date
aq_cleaned['Hour'] = aq_cleaned['Datetime'].dt.hour

# Daily average per country
daily_avg = aq_cleaned.groupby(['Country', 'Date', 'Notation'])['Value'].mean().reset_index()
daily_avg = daily_avg.rename(columns={'Value': 'Daily_Avg'})

# Daytime average (9-18 End time, i.e., 8-17 actual hour)
daytime_mask = aq_cleaned['Hour'].between(9, 18)
daytime_avg = aq_cleaned[daytime_mask].groupby(['Country', 'Date', 'Notation'])['Value'].mean().reset_index()
daytime_avg = daytime_avg.rename(columns={'Value': 'Daytime_Avg'})

# Rush-hour average (8-10 and 15-18 End time, i.e., 7-9 and 15-17 actual hour)
rush_mask = (aq_cleaned['Hour'].between(8, 10)) | (aq_cleaned['Hour'].between(15, 18))
rush_avg = aq_cleaned[rush_mask].groupby(['Country', 'Date', 'Notation'])['Value'].mean().reset_index()
rush_avg = rush_avg.rename(columns={'Value': 'RushHour_Avg'})

# Merge all averages into one DataFrame
aq_merged_cleaned = daily_avg.merge(daytime_avg, on=['Country', 'Date', 'Notation'], how='left')
aq_merged_cleaned = aq_merged_cleaned.merge(rush_avg, on=['Country', 'Date', 'Notation'], how='left')

# Save the cleaned dataset
aq_merged_cleaned.to_csv("../data/processed/AQ_merged_cleaned.csv", index=False)

print(aq_merged_cleaned.head())

  Country        Date    Notation  Daily_Avg  Daytime_Avg  RushHour_Avg
0      AT  2009-01-01         NO2  35.525558          NaN           NaN
1      AT  2013-01-01         CO2  35.851000    38.626760     37.535657
2      AT  2013-01-01          NO  80.641822    52.104975     51.446887
3      AT  2013-01-01         NO2  26.560859    22.000584     24.580176
4      AT  2013-01-01  NOX as NO2  53.923347    45.303470     44.564314


In [7]:
import pandas as pd
import numpy as np

# Load the merged air quality data
aq_df = pd.read_csv("../data/processed/AQ_merged_data_export.csv")

# Prepare datetime and helper columns
aq_df['Datetime'] = pd.to_datetime(aq_df['End'], errors='coerce')
aq_df = aq_df.dropna(subset=['Datetime'])
aq_df['Year'] = aq_df['Datetime'].dt.year
aq_df['Hour'] = aq_df['Datetime'].dt.hour
aq_df['Weekday'] = aq_df['Datetime'].dt.weekday  # 0=Monday, ..., 6=Sunday
aq_df['IsWeekend'] = aq_df['Weekday'] >= 5

# Helper masks
daytime_mask = aq_df['Hour'].between(9, 18)  # End time 9-18 (i.e., 8-17 actual hour)
rush_mask = (aq_df['Hour'].between(8, 10)) | (aq_df['Hour'].between(16, 18))  # End time 8-10, 15-18

def annual_avg(df, mask, period):
    # period: 'fullweek', 'weekday', 'weekend'
    if period == 'weekday':
        df = df[~df['IsWeekend']]
    elif period == 'weekend':
        df = df[df['IsWeekend']]
    df = df[mask]
    return (
        df.groupby(['Country', 'Notation', 'Year'])['Value']
        .mean()
        .reset_index()
        .rename(columns={'Value': f'AnnualAvg_{period}'})
    )

# Compute all combinations
results = []

for period in ['fullweek', 'weekday', 'weekend']:
    # Daytime
    dt_avg = annual_avg(aq_df, daytime_mask, period)
    dt_avg['Type'] = 'Daytime'
    results.append(dt_avg)
    # Rush-hour
    rh_avg = annual_avg(aq_df, rush_mask, period)
    rh_avg['Type'] = 'RushHour'
    results.append(rh_avg)

# Concatenate and pivot for clarity
annual_averages = pd.concat(results, ignore_index=True)
annual_averages = annual_averages.pivot_table(
    index=['Country', 'Notation', 'Year'],
    columns=['Type'],
    values=['AnnualAvg_fullweek', 'AnnualAvg_weekday', 'AnnualAvg_weekend']
).reset_index()

# Flatten columns
annual_averages.columns = ['_'.join([str(i) for i in col if i]) for col in annual_averages.columns.values]

# rename notation column to 'Pollutant'
annual_averages = annual_averages.rename(columns={'Notation': 'Pollutant'})

# Save to processed folder
annual_averages.to_csv("../data/processed/AQ_annual_averages.csv", index=False)

print(annual_averages.head())

C:\Users\jskif\AppData\Local\Temp\ipykernel_18920\3582149203.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[mask]
C:\Users\jskif\AppData\Local\Temp\ipykernel_18920\3582149203.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[mask]
C:\Users\jskif\AppData\Local\Temp\ipykernel_18920\3582149203.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[mask]
C:\Users\jskif\AppData\Local\Temp\ipykernel_18920\3582149203.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[mask]


  Country Pollutant  Year  AnnualAvg_fullweek_Daytime  \
0      AT       CO2  2013                   60.556376   
1      AT       CO2  2014                   56.194409   
2      AT       CO2  2015                   65.263546   
3      AT       CO2  2016                   61.306118   
4      AT       CO2  2017                   65.076009   

   AnnualAvg_fullweek_RushHour  AnnualAvg_weekday_Daytime  \
0                    54.747262                  58.785469   
1                    50.201177                  54.746705   
2                    58.792255                  62.895737   
3                    54.839356                  59.180094   
4                    59.085989                  63.561608   

   AnnualAvg_weekday_RushHour  AnnualAvg_weekend_Daytime  \
0                   52.816588                  65.000672   
1                   48.623885                  59.827592   
2                   56.234633                  71.205836   
3                   52.795785                  66.